In [225]:
import pandas as pd
from rdflib import Graph, Literal, RDF, URIRef, Namespace, BNode
from rdflib.namespace import XSD
import urllib.parse
import csv

In [226]:
#CSV Files:
coursecsv = 'CSVs/data.csv'
studentcsv = 'CSVs/students.csv'
lecturescsv = 'CSVs/lectures.csv'
topicscsv = 'CSVs/topics.csv'

# TTL Files: (Generated)
dataPath = 'Data/data.ttl'
coursePath = 'Data/courses.ttl'
studentPath = 'Data/students.ttl'
lecturesPath = 'Data/lectures.ttl'
topicsPath = 'Data/topics.ttl'

In [227]:
g = Graph()
ex = Namespace("http://example.org/vocab/")
foaf = Namespace("http://xmlns.com/foaf/0.1/")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
xsd = Namespace("http://www.w3.org/2001/XMLSchema#")
g.bind("ex", ex)
g.bind("foaf", foaf)
g.bind("rdf", rdf)
g.bind("rdfs", rdfs)
g.bind("xsd", xsd)

# Course Data (From Concordia Open Data):

In [228]:
catalog_df = pd.read_csv(coursecsv, encoding='utf-16')
for index, row in catalog_df.iterrows():
    course_id_str = str(row['Course ID'])
    course_uri = URIRef(f"http://example.org/vocab/course/{urllib.parse.quote(course_id_str)}")
    g.add((course_uri, RDF.type, ex.Course))
    g.add((course_uri, ex.subject, Literal(row['Subject'], datatype=XSD.string)))
    g.add((course_uri, ex.number, Literal(row['Catalog'], datatype=XSD.string)))
    g.add((course_uri, ex.description, Literal(row['Long Title'], datatype=XSD.string)))
    g.add((course_uri, ex.credits, Literal(row['Class Units'], datatype=XSD.decimal)))
    if not pd.isnull(row['Pre Requisite Description']):
        g.add((course_uri, ex.preRequisiteDescription, Literal(row['Pre Requisite Description'], datatype=XSD.string)))
    if not pd.isnull(row['Equivalent Courses']):
        g.add((course_uri, ex.equivalentCourses, Literal(row['Equivalent Courses'], datatype=XSD.string)))


g.serialize(destination=coursePath, format='turtle')

<Graph identifier=N061660d5a19a43e696efc1ed3034a31a (<class 'rdflib.graph.Graph'>)>

# Students

In [229]:
g = Graph()
ex = Namespace("http://example.org/vocab/")
foaf = Namespace("http://xmlns.com/foaf/0.1/")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
xsd = Namespace("http://www.w3.org/2001/XMLSchema#")
g.bind("ex", ex)
g.bind("foaf", foaf)
g.bind("rdf", rdf)
g.bind("rdfs", rdfs)
g.bind("xsd", xsd)

In [230]:
students_df = pd.read_csv(studentcsv)
for index, row in students_df.iterrows():
    student_uri = ex[f'student/{urllib.parse.quote(row["id_number"])}']
    g.add((student_uri, RDF.type, ex.Student))
    g.add((student_uri, foaf.name, Literal(f"{row['first_name']} {row['last_name']}", datatype=XSD.string)))
    g.add((student_uri, foaf.mbox, Literal(row['email'], datatype=XSD.string)))
    g.add((student_uri, ex.studentID, Literal(row['id_number'], datatype=XSD.string)))

    for i in range(1, 5):  
        course_uri = ex[f'course/{row[f"course_{i}_id"]}']
        grade = row[f"course_{i}_grade"]
        course_completion = BNode()
        g.add((student_uri, ex.completedCourse, course_completion))
        g.add((course_completion, RDF.type, URIRef("http://example.org/vocab/CompletedCourse")))
        g.add((course_completion, ex.course, course_uri))
        g.add((course_completion, ex.courseGrade, Literal(grade, datatype=XSD.string)))
    
    for comp_field in ['competent_in_course_id1', 'competent_in_course_id2']:
        competency_course_uri = ex[f'course/{row[comp_field]}']
        g.add((student_uri, ex.hasCompetency, competency_course_uri))


g.serialize(destination=studentPath, format='turtle')

<Graph identifier=N7d54c6fdde954bcf98d224038e1601ac (<class 'rdflib.graph.Graph'>)>

# Lecture

In [231]:
g = Graph()
ex = Namespace("http://example.org/vocab/")
foaf = Namespace("http://xmlns.com/foaf/0.1/")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
xsd = Namespace("http://www.w3.org/2001/XMLSchema#")
g.bind("ex", ex)
g.bind("foaf", foaf)
g.bind("rdf", rdf)
g.bind("rdfs", rdfs)
g.bind("xsd", xsd)

In [232]:
lectures_df = pd.read_csv(lecturescsv)
for index, row in lectures_df.iterrows():
    # Create a URI for the lecture based on its ID
    lecture_uri = URIRef(f"http://example.org/vocab/lecture/{urllib.parse.quote(str(row['LectureID']))}")
    
    g.add((lecture_uri, RDF.type, ex.Lecture))
    
    course_uri = URIRef(f"http://example.org/vocab/course/{urllib.parse.quote(str(row['CourseID']))}")
    g.add((lecture_uri, ex.isPartOfCourse, course_uri))
    
    g.add((lecture_uri, ex.lectureNumber, Literal(row['LectureNumber'], datatype=XSD.integer)))
    g.add((lecture_uri, ex.lectureName, Literal(row['LectureName'], datatype=XSD.string)))
    g.add((lecture_uri, ex.lectureContentLink, Literal(row['ContentLink'], datatype=XSD.anyURI)))
    g.add((lecture_uri, rdfs.seeAlso, URIRef(row['SeeAlsoLink'])))

g.serialize(destination=lecturesPath, format='turtle')


<Graph identifier=Nd45c97ac75334f3c877e3651c628f1bd (<class 'rdflib.graph.Graph'>)>

# Topics

In [233]:
g = Graph()
ex = Namespace("http://example.org/vocab/")
foaf = Namespace("http://xmlns.com/foaf/0.1/")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
xsd = Namespace("http://www.w3.org/2001/XMLSchema#")
g.bind("ex", ex)
g.bind("foaf", foaf)
g.bind("rdf", rdf)
g.bind("rdfs", rdfs)
g.bind("xsd", xsd)

In [234]:
topics_df = pd.read_csv(topicscsv)
for index, row in topics_df.iterrows():
    topic_uri = URIRef(f"http://example.org/vocab/topic/{urllib.parse.quote(str(row['TopicID']))}")
    g.add((topic_uri, RDF.type, ex.Topic))
    g.add((topic_uri, ex.topicName, Literal(row['TopicName'], datatype=XSD.string)))
    g.add((topic_uri, ex.topicProvenance, Literal(row['TopicProvenance'], datatype=XSD.string)))
    g.add((topic_uri, ex.topicLink, URIRef(row['TopicLink'])))
    lecture_uri = URIRef(f"http://example.org/vocab/lecture/{urllib.parse.quote(str(row['LectureID']))}")
    g.add((topic_uri, ex.isTopicOfLecture, lecture_uri))
    course_uri = URIRef(f"http://example.org/vocab/course/{urllib.parse.quote(str(row['CourseID']))}")
    g.add((topic_uri, ex.isTopicOfCourse, course_uri))

g.serialize(destination=topicsPath, format='turtle')

<Graph identifier=Nc02a4c2d84b6445fa61d20db17ef42eb (<class 'rdflib.graph.Graph'>)>